In [27]:
from typing import List

from openai import AzureOpenAI
import os
from dotenv import load_dotenv
load_dotenv()

model = AzureOpenAI(
    azure_endpoint=os.environ["AZURE_OPENAI_ENDPOINT"],
    api_key=os.environ["AZURE_OPENAI_API_KEY"],
    api_version=os.environ["AZURE_OPENAI_API_VERSION"],
)


# This is flexible, but you can define your agent here, or call your agent API here.
def my_chat_bot(messages: List[dict]) -> dict:
    system_message = {
        "role": "system",
        "content": "You are a customer support agent for an airline.",
    }
    messages = [system_message] + messages
    completion = model.chat.completions.create(
        messages=messages, model=os.environ["AZURE_OPENAI_DEPLOYMENT_NAME"]
    )
    return completion.choices[0].message.model_dump()

In [1]:
from typing import List

from openai import AzureOpenAI
import os
from dotenv import load_dotenv
load_dotenv()

model = AzureOpenAI(
    azure_endpoint=os.environ["AZURE_OPENAI_ENDPOINT"],
    api_key=os.environ["AZURE_OPENAI_API_KEY"],
    api_version=os.environ["AZURE_OPENAI_API_VERSION"],
)


# This is flexible, but you can define your agent here, or call your agent API here.
def my_chat_bot(messages: List[dict]) -> dict:
    system_message = {
        "role": "system",
        "content": "You are a qa engineer expert who can analyse inputs and provide automation scripts.",
    }
    messages = [system_message] + messages
    completion = model.chat.completions.create(
        messages=messages, model=os.environ["AZURE_OPENAI_DEPLOYMENT_NAME"]
    )
    return completion.choices[0].message.model_dump()

In [2]:
my_chat_bot([{"role": "user", "content": "hi!"}])

{'content': 'Hello! How can I assist you today? Do you have any specific questions or tasks related to automation testing that you need help with?',
 'refusal': None,
 'role': 'assistant',
 'annotations': None,
 'audio': None,
 'function_call': None,
 'tool_calls': None}

In [ ]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
# from langchain_openai import ChatOpenAI
from code_manager.llm import llm

system_prompt_template = """You are a customer of an airline company. \
You are interacting with a user who is a customer support person. \

{instructions}

When you are finished with the conversation, respond with a single word 'FINISHED'"""

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt_template),
        MessagesPlaceholder(variable_name="messages"),
    ]
)
instructions = """Your name is Harrison. You are trying to get a refund for the trip you took to Alaska. \
You want them to give you ALL the money back. \
This trip happened 5 years ago."""

prompt = prompt.partial(name="Harrison", instructions=instructions)

model = llm

simulated_user = prompt | model

In [9]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_openai import ChatOpenAI
from code_manager.llm import llm
import json

system_prompt_template = """You are a QA tester specializing in Java Selenium automation using the Cucumber framework. 
Your task is to converse with a chatbot as a user and provide feedback on its responses.
When you are finished with the conversation, respond with a single word 'FINISHED'"""
# system_prompt_template = """You are a customer of an airline company. \
# You are interacting with a user who is a customer support person. \

# {instructions}

# When you are finished with the conversation, respond with a single word 'FINISHED'"""


prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt_template),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

In [8]:
with open("page_prompt.txt", "r") as file:
    instructions = file.read()

with open('outputs/test_run.json','r') as f:
    test_run = json.load(f)

test_run['relevant_snippets']

[{'query': "Find a function that clicks on a button with the class '.btn.btn-default.add-to-cart' and text 'Add to cart'.",
  'snippets': [{'match_score': 0.9621464014053345,
    'lines': [45, 51],
    'node_type': 'method_declaration',
    'text': '@When("The user hovers over first product and clicks Add to cart")\npublic void the_user_hovers_over_first_product_and_clicks_add_to_cart() {\n    BrowserUtils.scrollToElement(productsPage.productsList.get(0));\n    BrowserUtils.hover(productsPage.productsList.get(0));\n    BrowserUtils.waitForClickablility(productsPage.addToCartButtons.get(0), 2);\n    productsPage.addToCartButtons.get(0).click();\n}',
    'path': '.\\Selenium_Cucumber_AutomationExercise\\src\\test\\java\\com\\myProject\\stepDefinitions\\Products_StepDefs.java',
    'model_name': 'Repo_Index'},
   {'match_score': 0.9322923421859741,
    'lines': [58, 63],
    'node_type': 'method_declaration',
    'text': '@When("The user hovers over second product and clicks Add to cart")

In [10]:
# instructions = instructions
# prompt = prompt.partial(user="QA Tester", instructions=instructions)

simulated_user = prompt | llm

In [11]:
from langchain_core.messages import HumanMessage

messages = [HumanMessage(content="Hi! How can I help you?")]
simulated_user.invoke({"messages": messages})

AIMessage(content="Hi! I'm a QA tester specializing in Java Selenium automation using the Cucumber framework. Can you help me with setting up a new project?", additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 29, 'prompt_tokens': 71, 'total_tokens': 100, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-2024-05-13', 'system_fingerprint': 'fp_65792305e4', 'prompt_filter_results': [{'prompt_index': 0, 'content_filter_results': {'hate': {'filtered': False, 'severity': 'safe'}, 'self_harm': {'filtered': False, 'severity': 'safe'}, 'sexual': {'filtered': False, 'severity': 'safe'}, 'violence': {'filtered': False, 'severity': 'safe'}}}], 'finish_reason': 'stop', 'logprobs': None, 'content_filter_results': {'hate': {'filtered': False, 'severity': 'safe'}, 'self_harm': {'filte

In [12]:
from langchain_community.adapters.openai import convert_message_to_dict
from langchain_core.messages import AIMessage


def chat_bot_node(state):
    messages = state["messages"]
    # Convert from LangChain format to the OpenAI format, which our chatbot function expects.
    messages = [convert_message_to_dict(m) for m in messages]
    # Call the chat bot
    chat_bot_response = my_chat_bot(messages)
    # Respond with an AI Message
    return {"messages": [AIMessage(content=chat_bot_response["content"])]}

In [13]:
def _swap_roles(messages):
    new_messages = []
    for m in messages:
        if isinstance(m, AIMessage):
            new_messages.append(HumanMessage(content=m.content))
        else:
            new_messages.append(AIMessage(content=m.content))
    return new_messages


def simulated_user_node(state):
    messages = state["messages"]
    # Swap roles of messages
    new_messages = _swap_roles(messages)
    # Call the simulated user
    response = simulated_user.invoke({"messages": new_messages})
    # This response is an AI message - we need to flip this to be a human message
    return {"messages": [HumanMessage(content=response.content)]}

In [14]:
def should_continue(state):
    messages = state["messages"]
    if len(messages) > 6:
        return "end"
    elif messages[-1].content == "FINISHED":
        return "end"
    else:
        return "continue"

In [15]:
from langgraph.graph import END, StateGraph, START
from langgraph.graph.message import add_messages
from typing import Annotated
from typing_extensions import TypedDict


class State(TypedDict):
    messages: Annotated[list, add_messages]


graph_builder = StateGraph(State)
graph_builder.add_node("user", simulated_user_node)
graph_builder.add_node("chat_bot", chat_bot_node)
# Every response from  your chat bot will automatically go to the
# simulated user
graph_builder.add_edge("chat_bot", "user")
graph_builder.add_conditional_edges(
    "user",
    should_continue,
    # If the finish criteria are met, we will stop the simulation,
    # otherwise, the virtual user's message will be sent to your chat bot
    {
        "end": END,
        "continue": "chat_bot",
    },
)
# The input will first go to your chat bot
graph_builder.add_edge(START, "chat_bot")
simulation = graph_builder.compile()

In [17]:
png_bytes = simulation.get_graph().draw_mermaid_png()
with open('static/llm_converse_workflow.png', 'wb') as file:
    file.write(png_bytes)

In [3]:
from code_manager.prompts import page_function_prompt
import json
with open('outputs/test_run.json','r') as f:
        test_run = json.load(f)

inputs = {
    "snippets": test_run['relevant_snippets'],
    "action_input": test_run['recorded_steps'],
    "testcase": test_run['testcase'],
    "feature_files": test_run['feature_files'],
    "project_files":test_run['project_files']
}

page_function_prompt.invoke(inputs).text

'You are an expert QA engineer specializing in Java Selenium automation using the Cucumber framework. Your task is to **generate or reuse page functions and return complete Java class files** based on the provided inputs.\n\n## Inputs:\n\n1. **testcase** – A set of steps describing the test case. - 1. Click on Add to cart button. \n2. Click on Continue Shopping button. \n3. Click on Products link. \n4. Enter text in search bar. \n5. Click on search button.\n2. **action_input** – Recorded actions corresponding to each test step. - [{\'id\': 1, \'performedAct\': \'click\', \'tagName\': \'A\', \'selector\': \'.btn.btn-default.add-to-cart\', \'text\': \'Add to cart\', \'timestamp\': \'03/27/2025, 05:48:17 PM\', \'tabUrl\': \'https://automationexercise.com/\', \'breakdownHTML\': {\'a\': [{\'attributes\': {\'data-product-id\': \'2\', \'class\': [\'btn\', \'btn-default\', \'add-to-cart\']}, \'content\': \'Add to cart\'}], \'i\': [{\'attributes\': {\'class\': [\'fa\', \'fa-shopping-cart\']}, \

In [18]:
for chunk in simulation.stream({"messages": []}):
    # Print out all events aside from the final end chunk
    if END not in chunk:
        print(chunk)
        print("===")

AttributeError: 'AzureChatOpenAI' object has no attribute 'chat'

In [32]:
from typing import List, Annotated
from typing_extensions import TypedDict

import os
import json
from dotenv import load_dotenv

from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_community.adapters.openai import convert_message_to_dict
from langchain_core.messages import AIMessage, HumanMessage, BaseMessage
from openai import AzureOpenAI

from langgraph.graph import END, StateGraph, START
from langgraph.graph.message import add_messages

from code_manager.llm import llm
from code_manager.prompts import page_function_prompt

from loguru import logger

# Load environment variables
load_dotenv()

# Configure logger
logger.add("logs/simulation.log", rotation="1 MB", retention="10 days", level="DEBUG")

# Initialize Azure OpenAI client
model = AzureOpenAI(
    azure_endpoint=os.environ["AZURE_OPENAI_ENDPOINT"],
    api_key=os.environ["AZURE_OPENAI_API_KEY"],
    api_version=os.environ["AZURE_OPENAI_API_VERSION"],
)

def my_chat_bot(messages: List[dict]) -> dict:
    # logger.debug(f"ChatBot received messages: {messages}")
    system_message = {
        "role": "system",
        "content": "You are a qa engineer expert who can analyse inputs and provide automation scripts.",
    }
    messages = [system_message] + messages
    completion = model.chat.completions.create(
        messages=messages, model=os.environ["AZURE_OPENAI_DEPLOYMENT_NAME"]
    )
    response = completion.choices[0].message.model_dump()
    logger.debug(f"ChatBot response: {response}")
    return response

system_prompt_template = """You are a senior QA reviewer working with a conversational AI assistant that generates automation code.
Your task is to simulate a QA engineer reviewing the assistant's response based on a provided test case, relevant HTML, and code snippets.
You must:
- Validate whether the Java Selenium Cucumber code generated by the assistant correctly reflects the test case intent.
- Ensure code follows best practices: clean structure, readable steps, Page Object usage where possible.
- If the answer is incorrect, give concise but constructive feedback and request a fix.
- If the response is correct and complete, simply respond with the word FINISHED.

Your tone should be collaborative but focused on code quality and correctness."""

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt_template),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

simulated_user = prompt | llm

def chat_bot_node(state):
    logger.info("chat_bot_node invoked")
    messages = state["messages"]
    messages = [convert_message_to_dict(m) for m in messages]
    chat_bot_response = my_chat_bot(messages)
    # logger.debug(f"chat_bot_node returning: {chat_bot_response['content']}")
    return {"messages": [AIMessage(content=chat_bot_response["content"])]}

def _swap_roles(messages: List[BaseMessage]):
    new_messages = []
    for m in messages:
        if isinstance(m, AIMessage):
            new_messages.append(HumanMessage(content=m.content))
        else:
            new_messages.append(AIMessage(content=m.content))
    return new_messages

def simulated_user_node(state):
    logger.info("simulated_user_node invoked")
    messages = state["messages"]
    new_messages = _swap_roles(messages)
    response = simulated_user.invoke({"messages": new_messages})
    logger.debug(f"simulated_user_node response: {response.content}")
    return {"messages": [HumanMessage(content=response.content)]}

def should_continue(state):
    messages = state["messages"]
    if len(messages) > 6:
        logger.info("Stopping: Max message limit reached.")
        return "end"
    elif messages[-1].content == "FINISHED":
        logger.info("Stopping: Conversation finished.")
        return "end"
    else:
        return "continue"

class State(TypedDict):
    messages: Annotated[list, add_messages]

graph_builder = StateGraph(State)
graph_builder.add_node("user", simulated_user_node)
graph_builder.add_node("chat_bot", chat_bot_node)

graph_builder.add_edge("chat_bot", "user")
graph_builder.add_conditional_edges(
    "user",
    should_continue,
    {
        "end": END,
        "continue": "chat_bot",
    },
)
graph_builder.add_edge(START, "chat_bot")
simulation = graph_builder.compile()

# Generate and save the graph visualization
png_bytes = simulation.get_graph().draw_mermaid_png()
os.makedirs("static", exist_ok=True)
with open('static/llm_converse_workflow.png', 'wb') as file:
    file.write(png_bytes)
logger.info("Saved workflow graph to static/llm_converse_workflow.png")

if __name__ == "__main__":
    logger.info("Starting simulation...")
    try:
        with open('outputs/test_run.json','r') as f:
            test_run = json.load(f)
            # logger.debug(f"Loaded test run: {test_run}")
    except Exception as e:
        logger.error(f"Failed to load test run: {e}")
        raise

    inputs = {
        "snippets": test_run['relevant_snippets'],
        "action_input": test_run['recorded_steps'],
        "testcase": test_run['testcase'],
        "feature_files": test_run['feature_files'],
        "project_files": test_run['project_files'][2]
    }

    try:
        initial_message = page_function_prompt.invoke(inputs).text
        # logger.debug(f"Initial message to simulation: {initial_message}")
    except Exception as e:
        logger.error(f"Failed to invoke page_function_prompt: {e}")
        raise

    # for chunk in simulation.stream({"messages": [initial_message]}):
    #     if END not in chunk:
    #         logger.info(f"Simulation step result: {chunk}")
    #         print(chunk)
    #         print("===")

    simulation_result = simulation.invoke({"messages": [initial_message]})

    

    logger.info("Simulation finished.")


2025-04-07 12:41:19.635 | INFO     | __main__:<module>:127 - Saved workflow graph to static/llm_converse_workflow.png
2025-04-07 12:41:19.639 | INFO     | __main__:<module>:130 - Starting simulation...
2025-04-07 12:41:19.653 | INFO     | __main__:chat_bot_node:68 - chat_bot_node invoked
2025-04-07 12:41:35.043 | DEBUG    | __main__:my_chat_bot:45 - ChatBot response: {'content': '```json\n{\n  "page_functions": {\n    "SearchProductPage.java": "package com.myProject.pages;\\n\\nimport org.openqa.selenium.By;\\nimport org.openqa.selenium.WebDriver;\\nimport org.openqa.selenium.WebElement;\\nimport org.openqa.selenium.support.FindBy;\\nimport org.openqa.selenium.support.PageFactory;\\nimport com.myProject.utilities.Driver;\\n\\npublic class SearchProductPage {\\n    public SearchProductPage() {\\n        PageFactory.initElements(Driver.get(), this);\\n    }\\n\\n    @FindBy(css = \\".btn.btn-default.add-to-cart\\")\\n    public WebElement addToCartButton;\\n\\n    @FindBy(css = \\".btn.b

In [33]:
simulation_result['messages'][-2]

AIMessage(content='```json\n{\n  "page_functions": {\n    "SearchProductPage.java": "package com.myProject.pages;\\n\\nimport org.openqa.selenium.WebElement;\\nimport org.openqa.selenium.support.FindBy;\\nimport org.openqa.selenium.support.PageFactory;\\nimport com.myProject.utilities.Driver;\\n\\npublic class SearchProductPage {\\n    public SearchProductPage() {\\n        PageFactory.initElements(Driver.get(), this);\\n    }\\n\\n    @FindBy(css = \\".btn.btn-default.add-to-cart\\")\\n    public WebElement addToCartButton;\\n\\n    @FindBy(css = \\".btn.btn-success.close-modal.btn-block\\")\\n    public WebElement continueShoppingButton;\\n\\n    @FindBy(linkText = \\"î£¸ Products\\")\\n    public WebElement productsLink;\\n\\n    @FindBy(id = \\"search_product\\")\\n    public WebElement searchInput;\\n\\n    @FindBy(css = \\".fa.fa-search\\")\\n    public WebElement searchButton;\\n\\n    public void clickProductsLink() {\\n        productsLink.click();\\n    }\\n\\n    public void

In [34]:
x = simulation_result['messages'][-2].content

In [ ]:
import mdextractor

markdown_text = x

code_blocks = mdextractor.extract_md_blocks(markdown_text)

In [38]:
import json
b = json.loads(code_blocks[0])
print(b['page_functions']['SearchProductPage.java'])

package com.myProject.pages;

import org.openqa.selenium.WebElement;
import org.openqa.selenium.support.FindBy;
import org.openqa.selenium.support.PageFactory;
import com.myProject.utilities.Driver;

public class SearchProductPage {
    public SearchProductPage() {
        PageFactory.initElements(Driver.get(), this);
    }

    @FindBy(css = ".btn.btn-default.add-to-cart")
    public WebElement addToCartButton;

    @FindBy(css = ".btn.btn-success.close-modal.btn-block")
    public WebElement continueShoppingButton;

    @FindBy(linkText = "î£¸ Products")
    public WebElement productsLink;

    @FindBy(id = "search_product")
    public WebElement searchInput;

    @FindBy(css = ".fa.fa-search")
    public WebElement searchButton;

    public void clickProductsLink() {
        productsLink.click();
    }

    public void enterSearchText(String searchText) {
        searchInput.sendKeys(searchText);
    }

    public void clickSearchButton() {
        searchButton.click();
    }
}


#### Step Function Generation

In [2]:
from typing import List, Annotated
from typing_extensions import TypedDict

import os
import json
from dotenv import load_dotenv

from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_community.adapters.openai import convert_message_to_dict
from langchain_core.messages import AIMessage, HumanMessage, BaseMessage
from openai import AzureOpenAI

from langgraph.graph import END, StateGraph, START
from langgraph.graph.message import add_messages

from code_manager.llm import llm
from code_manager.prompts import step_function_prompt

from loguru import logger

# Load environment variables
load_dotenv()

# Initialize Azure OpenAI client
model = AzureOpenAI(
    azure_endpoint=os.environ["AZURE_OPENAI_ENDPOINT"],
    api_key=os.environ["AZURE_OPENAI_API_KEY"],
    api_version=os.environ["AZURE_OPENAI_API_VERSION"],
)

def my_chat_bot(messages: List[dict]) -> dict:
    # logger.debug(f"ChatBot received messages: {messages}")
    system_message = {
        "role": "system",
        "content": "You are a qa engineer expert who can analyse inputs and provide automation scripts. Always respond in JSON format as specified in initial message.",
    }
    messages = [system_message] + messages
    completion = model.chat.completions.create(
        messages=messages, model=os.environ["AZURE_OPENAI_DEPLOYMENT_NAME"]
    )
    response = completion.choices[0].message.model_dump()
    logger.debug(f"ChatBot response: {response}")
    return response

system_prompt_template = """
You are a senior QA reviewer working with a conversational AI assistant that generates Cucumber step definitions for a Java Selenium automation project.

Your job is to simulate a QA engineer reviewing the assistant's generated step functions based on:
- A provided feature file (gherkin)
- Existing code snippets and reusable functions from the codebase
- Generated page function files from the assistant previously.

Your responsibilities:

- Ensure each generated step function accurately represents a step from the feature file.
- Confirm the correct use of Cucumber annotations: @Given, @When, @Then.
- Validate that the step function **calls appropriate Page Object methods**, not raw WebDriver code.
- Check method and argument naming for clarity and consistency in both reused page functions and newly generated page functions.
- Review imports and class structure for correctness.
- Detect any steps that are:
    - Missing
    - Incorrectly mapped
    - Duplicated
- Ensure that each gherkin of the feature file is represented by a step function either in the generated code or in the existing code snippets.
- All methods are utilised correctly from the page function files.
- If everything is correct and complete, simply respond with: `FINISHED`.


 **IMPORTANT**:  
If any of the generated annotations (e.g., `@Given("user logs in")`) are already present in the relevant code snippets:
-  Do **not** regenerate that step.
-  Recommend removing it and do not mention it as it is already in repo and will be triggered.

 Provide **concise, constructive feedback** if something needs to be fixed or improved.
Do not write `FINISHED` if the response is incorrect or incomplete by the chatbot.


Be collaborative, but focused on code quality, correctness, reuse, and maintainability.
"""


prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt_template),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

simulated_user = prompt | llm

def chat_bot_node(state):
    logger.info("chat_bot_node invoked")
    messages = state["messages"]
    messages = [convert_message_to_dict(m) for m in messages]
    chat_bot_response = my_chat_bot(messages)
    # logger.debug(f"chat_bot_node returning: {chat_bot_response['content']}")
    return {"messages": [AIMessage(content=chat_bot_response["content"])]}

def _swap_roles(messages: List[BaseMessage]):
    new_messages = []
    for m in messages:
        if isinstance(m, AIMessage):
            new_messages.append(HumanMessage(content=m.content))
        else:
            new_messages.append(AIMessage(content=m.content))
    return new_messages

def simulated_user_node(state):
    logger.info("simulated_user_node invoked")
    messages = state["messages"]
    new_messages = _swap_roles(messages)
    response = simulated_user.invoke({"messages": new_messages})
    logger.debug(f"simulated_user_node response: {response.content}")
    return {"messages": [HumanMessage(content=response.content)]}

def should_continue(state):
    messages:List[BaseMessage] = state["messages"]
    if len(messages) > 6:
        logger.info("Stopping: Max message limit reached.")
        return "end"
    elif "FINISHED" in messages[-1].content.upper():
        logger.info("Stopping: Conversation finished.")
        return "end"
    else:
        return "continue"

class State(TypedDict):
    messages: Annotated[list, add_messages]

graph_builder = StateGraph(State)
graph_builder.add_node("user", simulated_user_node)
graph_builder.add_node("chat_bot", chat_bot_node)

graph_builder.add_edge("chat_bot", "user")
graph_builder.add_conditional_edges(
    "user",
    should_continue,
    {
        "end": END,
        "continue": "chat_bot",
    },
)
graph_builder.add_edge(START, "chat_bot")
simulation = graph_builder.compile()

# Generate and save the graph visualization
# png_bytes = simulation.get_graph().draw_mermaid_png()
# os.makedirs("static", exist_ok=True)
# with open('static/llm_converse_workflow.png', 'wb') as file:
#     file.write(png_bytes)
# logger.info("Saved workflow graph to static/llm_converse_workflow.png")


async def react_page(initial_message: str):
    logger.info("React page function invoked")
    simulation_result = await simulation.ainvoke({"messages": [initial_message]})
    logger.debug(f"Simulation result: {simulation_result}")
    return simulation_result


if __name__ == "__main__":
    logger.info("Starting simulation...")
    try:
        with open('outputs/test_run.json','r') as f:
            test_run = json.load(f)
            # logger.debug(f"Loaded test run: {test_run}")
    except Exception as e:
        logger.error(f"Failed to load test run: {e}")
        raise

    inputs = {
        "snippets": test_run['relevant_snippets'],
        "page_function_file": test_run['page_functions'],
        # "testcase": test_run['testcase'],
        "feature_files": test_run['feature_files'],
        "project_files": test_run['project_files'][1]
    }

    try:
        initial_message = step_function_prompt.invoke(inputs).text
        # logger.debug(f"Initial message to simulation: {initial_message}")
    except Exception as e:
        logger.error(f"Failed to invoke step_function_prompt: {e}")
        raise

    # for chunk in simulation.stream({"messages": [initial_message]}):
    #     if END not in chunk:
    #         logger.info(f"Simulation step result: {chunk}")
    #         print(chunk)
    #         print("===")

    simulation_result = simulation.invoke({"messages": [initial_message]})

    

    logger.info("Simulation finished.")


2025-04-07 14:05:33.876 | INFO     | __main__:<module>:162 - Starting simulation...
2025-04-07 14:05:33.883 | INFO     | __main__:chat_bot_node:92 - chat_bot_node invoked
2025-04-07 14:05:42.080 | DEBUG    | __main__:my_chat_bot:42 - ChatBot response: {'content': '```json\n{\n    "step_function": "package com.myProject.stepDefinitions;\\n\\nimport com.myProject.pages.ProductsPage;\\nimport com.myProject.pages.SearchProductPage;\\nimport com.myProject.utilities.BrowserUtils;\\nimport io.cucumber.java.en.And;\\nimport io.cucumber.java.en.When;\\n\\npublic class SearchProduct_StepDefs {\\n\\n    ProductsPage productsPage = new ProductsPage();\\n    SearchProductPage searchProductPage = new SearchProductPage(Driver.get());\\n\\n    @When(\\"The user hovers over first product and clicks Add to cart\\")\\n    public void the_user_hovers_over_first_product_and_clicks_add_to_cart() {\\n        BrowserUtils.scrollToElement(productsPage.productsList.get(0));\\n        BrowserUtils.hover(products

In [ ]:
import mdextractor
print(json.loads(mdextractor.extract_md_blocks(simulation_result['messages'][-2].content)[-1])['step_function'])

package com.myProject.stepDefinitions;

import com.myProject.pages.ProductsPage;
import com.myProject.pages.SearchProductPage;
import com.myProject.utilities.BrowserUtils;
import com.myProject.utilities.Driver;
import io.cucumber.java.en.And;
import io.cucumber.java.en.When;

public class SearchProduct_StepDefs {

    ProductsPage productsPage = new ProductsPage();
    SearchProductPage searchProductPage = new SearchProductPage(Driver.get());

    @And("The user clicks on Products link")
    public void the_user_clicks_on_products_link() {
        productsPage.clickBrandName("Products");
    }

    @When("The user enters product name in search input and click search button")
    public void the_user_enters_product_name_in_search_input_and_click_search_button() {
        searchProductPage.enterProductNameInSearch("Kids");
    }

    @And("The user clicks on search button")
    public void the_user_clicks_on_search_button() {
        BrowserUtils.clickWithJS(searchProductPage.searchButto

In [1]:
feature = '''@C134207
Scenario Outline: C134207_SmokeTestCreateWithDrawalDitem
Given User should login to the application and impersonate into an agent "<SheetName>" , <RowNumber>
When User is on the account wizard tab and search the client
And Select request and type in Account Management
When User clicks on complete and print button
And User select withdrawal method and click on continue button
Then User navigates to docu sign page and clicks on Continue & Finish button
And Fetch the tracking id
Then User upload the PDF file and validate the uploaded file'''

In [2]:
feature

'@C134207\nScenario Outline: C134207_SmokeTestCreateWithDrawalDitem\nGiven User should login to the application and impersonate into an agent "<SheetName>" , <RowNumber>\nWhen User is on the account wizard tab and search the client\nAnd Select request and type in Account Management\nWhen User clicks on complete and print button\nAnd User select withdrawal method and click on continue button\nThen User navigates to docu sign page and clicks on Continue & Finish button\nAnd Fetch the tracking id\nThen User upload the PDF file and validate the uploaded file'

In [3]:
import json
with open("outputs/test1_output.json",'r') as f:
    state = json.load(f)

In [4]:
snippets = state['relevant_snippets']
feature_files = state['feature_files']

In [5]:
feature_files[0]['code'] = feature

In [6]:
feature_files

[{'code': '@C134207\nScenario Outline: C134207_SmokeTestCreateWithDrawalDitem\nGiven User should login to the application and impersonate into an agent "<SheetName>" , <RowNumber>\nWhen User is on the account wizard tab and search the client\nAnd Select request and type in Account Management\nWhen User clicks on complete and print button\nAnd User select withdrawal method and click on continue button\nThen User navigates to docu sign page and clicks on Continue & Finish button\nAnd Fetch the tracking id\nThen User upload the PDF file and validate the uploaded file',
  'file_name': 'signup.feature',
  'file_path': 'repo_clone_dir/Selenium_Cucumber_AutomationExercise/src/test/resources/features',
  'file_description': 'Feature file for the signup test case, detailing the steps to navigate to the signup page, enter user details, and submit the form.',
  'file_type': 'feature'}]

In [7]:
gherkin_files_dict = {}
for i,file in enumerate(feature_files):
    gherkin_files_dict[f'{i}'] = file['code']
gherkin_files_dict

{'0': '@C134207\nScenario Outline: C134207_SmokeTestCreateWithDrawalDitem\nGiven User should login to the application and impersonate into an agent "<SheetName>" , <RowNumber>\nWhen User is on the account wizard tab and search the client\nAnd Select request and type in Account Management\nWhen User clicks on complete and print button\nAnd User select withdrawal method and click on continue button\nThen User navigates to docu sign page and clicks on Continue & Finish button\nAnd Fetch the tracking id\nThen User upload the PDF file and validate the uploaded file'}

In [42]:
# Parsing logic
gherkin_data = []

for key, content in gherkin_files_dict.items():
    if isinstance(content, str):
        lines = content.splitlines()
    else:
        lines = content  # if already a list

    current_scenario = None
    steps = []

    for idx, line in enumerate(lines):
        stripped = line.strip()
        lowered = stripped.lower()

        # Detect scenario or scenario outline
        if lowered.startswith("scenario:") or lowered.startswith("scenario outline:"):
            if current_scenario and steps:
                gherkin_data.append({
                    "scenario": current_scenario,
                    "steps": steps
                })
            current_scenario = stripped
            steps = []

        elif lowered.startswith(("given", "when", "then", "and")):
            steps.append(stripped)

    if current_scenario and steps:
        gherkin_data.append({
            "scenario": current_scenario,
            "steps": steps
        })

gherkin_data

[{'scenario': 'Scenario Outline: C134207_SmokeTestCreateWithDrawalDitem',
  'steps': ['Given User should login to the application and impersonate into an agent "<SheetName>" , <RowNumber>',
   'When User is on the account wizard tab and search the client',
   'And Select request and type in Account Management',
   'When User clicks on complete and print button',
   'And User select withdrawal method and click on continue button',
   'Then User navigates to docu sign page and clicks on Continue & Finish button',
   'And Fetch the tracking id',
   'Then User upload the PDF file and validate the uploaded file']}]

In [8]:
def read_scenarios(gherkin_files_dict:dict):
    gherkin_files_dict = []
    gherkin_scenarios = []
    print(gherkin_files_dict)
    previous_start_with = None

    for filename, content in gherkin_files_dict.items():
        # If content is a string, split it into lines
        if isinstance(content, str):
            lines = content.splitlines()
        else:
            lines = content  # assume it's already a list of lines
        
        for line in lines:
            if line.strip().lower().startswith("given"):
                previous_start_with = "Given"
                gherkin_scenarios.append(line.strip())
            elif line.strip().lower().startswith("when"):
                previous_start_with = "When"
                gherkin_scenarios.append(line.strip())
            elif line.strip().lower().startswith("then"):
                previous_start_with = "Then"
                gherkin_scenarios.append(line.strip())
            elif line.strip().lower().startswith("and"):
                line = previous_start_with + line.strip()[4:]
                gherkin_scenarios.append(line.strip())
    return gherkin_scenarios
      

In [9]:
read_scenarios(gherkin_files_dict)

[]


AttributeError: 'list' object has no attribute 'items'

In [4]:
from code_manager.utils import read_scenarios

read_scenarios(feature_files_generated)

AttributeError: 'list' object has no attribute 'items'